In [65]:
from sklearn.externals import joblib
import pandas as pd
import ast
import numpy as np
from xgboost import XGBClassifier

/home/aditya15007/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
tst_labels = pd.read_csv('test_variants.csv')
#print(tr_labels.head)

tst_txt = pd.read_csv('test_text.csv', sep='\|\|', skiprows=1, header=None,names=["ID","Text"])
#print(tr_txt.head)

test = pd.merge(tst_txt, tst_labels, on='ID')
#print(train.head)

test.describe(include='all')

/home/aditya15007/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,ID,Text,Gene,Variation
count,986.000000,986,986,986
unique,NaN,874,279,945
top,NaN,Among the best-studied therapeutic targets in ...,TP53,Truncating Mutations
freq,NaN,24,40,18
mean,493.500000,NaN,NaN,NaN
std,284.777984,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN
25%,247.250000,NaN,NaN,NaN
50%,493.500000,NaN,NaN,NaN
75%,739.750000,NaN,NaN,NaN


In [5]:
def load_preprocessed(filename):
    X_pre = []
    print("Loading pre-processed data...")
    with open(filename,'r') as f:
        arr = f.readlines()
        counter = 0
        for line in arr:
            counter+=1
            #print(counter)
            x = ast.literal_eval(line.strip("\n"))
            X_pre.append(str(x))
    new_text = pd.DataFrame({'preprocessed_txt': X_pre})
    return new_text
    #print(new_text)

In [68]:
#reference: http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/

class MeanW2VecVectorizer(object):
    def __init__(self, w2v):
        self.w2v = w2v
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(w2v)
        
    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [8]:
new_test_pre = load_preprocessed('new_test.txt')
new_test = test.join(new_test_pre)
print(new_test.head)

Loading pre-processed data...
<bound method NDFrame.head of       ID                                               Text      Gene  \
0      1  The incidence of breast cancer is increasing i...     CHEK2   
1      2  An unselected series of 310 colorectal carcino...     AXIN2   
2      3  Mycosis fungoides and Sézary syndrome are prim...      WNT4   
3      4   Regulated progression through the cell cycle ...    SUCLA2   
4      5  Pilocytic astrocytoma (PA) is emerging as a tu...      BRAF   
5      6  The nuclei that laboratories solution p53 KIT ...     CHEK2   
6      7  Myeloid differentiation 88 (MyD88) is the key ...     CHST3   
7      8  Human ESCCs 2 occur frequently worldwide (1) ....      RNF6   
8      9  large were of activity growth this product tol...     SPAST   
9     10  The protein kinase v-akt murine thymoma viral ...      AKT1   
10    11  Endometrial carcinoma is the most common gynec...     SCN4A   
11    12  Gastrointestinal stromal tumors (GIST) are cha...     

In [63]:
def create_sub_file(probs, filename):
    new_probs = pd.DataFrame(probs, columns =['class1','class2','class3','class4','class5','class6','class7','class8','class9'])
    #print(new_probs)
    new_ids = np.arange(1,probs.shape[0]+1)
    new_ids_df = pd.DataFrame(new_ids, columns=['ID'])
    #print(new_ids_df)
    new_df = new_ids_df.join(new_probs)
    print(new_df.head)
    new_df.to_csv(filename,index=False)

In [14]:
model = joblib.load('svm_linear.pkl')
print(model)
# prepro = meanW2V_vec_trtest.transform(sentences_test)
prepro = joblib.load('tf_idf_vectorizer.pkl').transform(new_test['preprocessed_txt'])
Y_preds = []
print("Predicting!")
for x in prepro:
    Y_preds.append(model.predict_proba(x))
Y_preds = np.array(Y_preds)
print(Y_preds.shape)

SVC(C=0.3, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting!
(986, 1, 9)


In [25]:
print(Y_preds.reshape(Y_preds.shape[0],Y_preds.shape[2]).shape)

(986, 9)


In [62]:
create_sub_file(Y_preds.reshape(Y_preds.shape[0],Y_preds.shape[2]),'svm_linear_tfidf.csv')

       class1    class2    class3    class4    class5    class6    class7  \
0    0.342127  0.054847  0.030347  0.406947  0.045987  0.079827  0.034543   
1    0.514999  0.059638  0.013270  0.206909  0.036237  0.038744  0.121144   
2    0.110585  0.250094  0.014410  0.084682  0.040155  0.048647  0.442129   
3    0.084871  0.245546  0.009870  0.034749  0.024655  0.018417  0.567092   
4    0.013087  0.150749  0.010738  0.019905  0.024814  0.014396  0.761071   
5    0.374429  0.043662  0.009486  0.411443  0.039893  0.034633  0.079269   
6    0.020189  0.122886  0.010102  0.025966  0.022856  0.023696  0.767786   
7    0.364833  0.097213  0.008508  0.314459  0.040121  0.082851  0.081681   
8    0.416888  0.016983  0.009026  0.470367  0.032326  0.035323  0.012181   
9    0.013638  0.049779  0.054538  0.013601  0.058003  0.012244  0.782432   
10   0.126503  0.088497  0.016930  0.608413  0.048955  0.042030  0.054116   
11   0.014211  0.734961  0.017866  0.009093  0.023628  0.023372  0.171309   

In [64]:
model = joblib.load('rand_for.pkl')
print(model)
# prepro = meanW2V_vec_trtest.transform(sentences_test)
Y_preds = []
print("Predicting!")
for x in prepro:
    Y_preds.append(model.predict_proba(x))
Y_preds = np.array(Y_preds)
print(Y_preds.shape)
create_sub_file(Y_preds.reshape(Y_preds.shape[0],Y_preds.shape[2]),'rand_for_tfidf.csv')

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Predicting!
(986, 1, 9)
<bound method NDFrame.head of       ID    class1    class2    class3    class4    class5    class6  \
0      1  0.214112  0.173325  0.024066  0.213329  0.050244  0.064786   
1      2  0.268082  0.105726  0.016305  0.256774  0.046895  0.042475   
2      3  0.142808  0.202523  0.020103  0.119365  0.037847  0.041014   
3      4  0.176835  0.175453  0.017442  0.152708  0.040785  0.041258   
4      5  0.139616  0.152858  0.021952  0.161617  0.052855  0.037662   
5      6  0.262002  0.107421  0.015566  0.280879  0.046128  0.044149   
6      7  0.088962  0

In [ ]:
model = joblib.load('rand_for_w2v.pkl')
print(model)
meanw2v_train = joblib.load('meanw2v_train.pkl')
sentences_test = joblib.load('sentences_test.pkl')
prepro_new = meanw2v_train.transform(sentences_test)
print("Predicting!")

In [76]:
Y_preds = model.predict_proba(prepro_new)
Y_preds = np.array(Y_preds)
print(Y_preds.shape)
create_sub_file(Y_preds,'rand_for_w2v_tr.csv')

(986, 9)
<bound method NDFrame.head of       ID    class1    class2    class3    class4    class5    class6  \
0      1  0.260107  0.211884  0.035432  0.189075  0.049003  0.103788   
1      2  0.365199  0.081095  0.015411  0.229634  0.018946  0.021959   
2      3  0.162174  0.235215  0.010245  0.152696  0.034344  0.048055   
3      4  0.300442  0.151337  0.007346  0.178519  0.014389  0.036979   
4      5  0.073927  0.123042  0.008643  0.090503  0.044781  0.036484   
5      6  0.471419  0.055831  0.000299  0.277037  0.014417  0.024096   
6      7  0.031680  0.121998  0.022220  0.042519  0.030006  0.028860   
7      8  0.308617  0.110632  0.011396  0.217278  0.034748  0.049690   
8      9  0.237743  0.048898  0.006205  0.539461  0.010403  0.015029   
9     10  0.016178  0.045262  0.145943  0.028366  0.022014  0.014477   
10    11  0.198476  0.090303  0.018930  0.450853  0.024525  0.026893   
11    12  0.014888  0.875176  0.004364  0.005031  0.002818  0.004178   
12    13  0.570209  0.014

In [77]:
Y_preds = joblib.load('svm_linear_w2v.pkl').predict_proba(prepro_new)
Y_preds = np.array(Y_preds)
print(Y_preds.shape)
create_sub_file(Y_preds,'svm_linear_w2v_tr.csv')

(986, 9)
<bound method NDFrame.head of       ID    class1    class2    class3    class4    class5    class6  \
0      1  0.204083  0.046685  0.166463  0.172144  0.035093  0.303666   
1      2  0.559712  0.031538  0.003056  0.267536  0.015757  0.021147   
2      3  0.121499  0.243451  0.014041  0.138184  0.049900  0.063498   
3      4  0.122327  0.311958  0.002368  0.013284  0.005755  0.020552   
4      5  0.009524  0.176851  0.006562  0.032951  0.033493  0.005851   
5      6  0.671985  0.009869  0.002198  0.264382  0.013169  0.013803   
6      7  0.087249  0.164832  0.012216  0.072675  0.062430  0.065551   
7      8  0.457759  0.126717  0.004802  0.062434  0.042992  0.085663   
8      9  0.368471  0.017037  0.004908  0.539778  0.009866  0.014454   
9     10  0.013583  0.017256  0.107016  0.012719  0.082886  0.020358   
10    11  0.262335  0.007685  0.018299  0.612046  0.036176  0.012138   
11    12  0.009418  0.571814  0.002833  0.006589  0.004214  0.046286   
12    13  0.656658  0.005

In [78]:
meanw2v_trtest = joblib.load('w2v_trtest.pkl')
Y_preds = joblib.load('rand_for_w2v_trtest.pkl').predict_proba(prepro_new)
Y_preds = np.array(Y_preds)
print(Y_preds.shape)
create_sub_file(Y_preds,'rand_for_w2v_trtest.csv')

(986, 9)
<bound method NDFrame.head of       ID    class1    class2    class3    class4    class5    class6  \
0      1  0.170662  0.147347  0.020070  0.215854  0.088226  0.147005   
1      2  0.152814  0.143325  0.027777  0.216396  0.083133  0.154084   
2      3  0.157822  0.179754  0.027790  0.209664  0.082404  0.119708   
3      4  0.149128  0.184519  0.024325  0.178523  0.093333  0.133165   
4      5  0.151738  0.167004  0.019690  0.188585  0.081785  0.126624   
5      6  0.159113  0.175086  0.025685  0.173191  0.079820  0.120845   
6      7  0.163577  0.160254  0.024347  0.204900  0.090813  0.126188   
7      8  0.141290  0.179784  0.021735  0.212992  0.082345  0.146266   
8      9  0.167126  0.150178  0.024779  0.204693  0.085166  0.143552   
9     10  0.140902  0.198227  0.026825  0.188169  0.079260  0.116190   
10    11  0.155295  0.159380  0.023363  0.187416  0.085071  0.155264   
11    12  0.143545  0.162945  0.022510  0.198231  0.082272  0.146295   
12    13  0.169128  0.160

In [79]:
Y_preds = joblib.load('logistic_w2v_trtest.pkl').predict_proba(prepro_new)
Y_preds = np.array(Y_preds)
print(Y_preds.shape)
create_sub_file(Y_preds,'logistic_w2v_trtest.csv')

(986, 9)
<bound method NDFrame.head of       ID    class1    class2    class3    class4    class5    class6  \
0      1  0.002292  0.000206  0.200572  0.002672  0.164609  0.458317   
1      2  0.000039  0.000517  0.241829  0.000059  0.194847  0.366578   
2      3  0.000584  0.005127  0.162213  0.018515  0.187733  0.380877   
3      4  0.000131  0.002836  0.316793  0.001502  0.172105  0.282221   
4      5  0.000184  0.003943  0.207941  0.000413  0.054542  0.367381   
5      6  0.000046  0.000091  0.310211  0.000125  0.168429  0.259525   
6      7  0.000586  0.003283  0.133633  0.005214  0.194846  0.304207   
7      8  0.000002  0.000844  0.113148  0.000579  0.359791  0.349727   
8      9  0.003453  0.003240  0.031947  0.009867  0.034293  0.388042   
9     10  0.002205  0.038431  0.267477  0.000307  0.017720  0.248415   
10    11  0.000074  0.003452  0.242560  0.000472  0.131690  0.281811   
11    12  0.002360  0.001142  0.271196  0.002183  0.228835  0.268286   
12    13  0.000005  0.000